In [822]:
#train funLearn model

In [823]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras.layers import (Activation, BatchNormalization, Conv2D, Dense,
                          Dropout, Flatten, MaxPooling2D)
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import tensorflowjs as tfjs

In [824]:
print(tf.__version__)

2.4.0


In [825]:
import sys
print("Python version")
print (sys.version)

Python version
3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [826]:
print(tfjs.__version__)

2.8.3


In [827]:
IMG_WIDTH=224
IMG_HEIGHT=224
img_folder=r'C:/Users/younis3/Desktop/train_intel/animals-archive/seg_train/'

In [828]:

# Define constants
IMG_HEIGHT = 80
IMG_WIDTH  = 80
CHANNELS   = 3
batch_size = 4

#train_path = 'C:/Users/younis3/Desktop/train_intel/archive/seg_train/seg_train/'
#test_path  = 'C:/Users/younis3/Desktop/train_intel/archive/seg_test/seg_test/'
#class_names = ['cat', 'chicken', 'duck', 'sheep', 'k','kk']

train_path = 'C:/Users/younis3/Desktop/train_intel/animals-archive/seg_train/'
test_path  = 'C:/Users/younis3/Desktop/train_intel/animals-archive/seg_test/'
class_names = ['cat', 'chicken', 'duck', 'sheep']

In [829]:
## Generators ##
 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1
)

# Training
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training'
)

# Validation
validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    subset='validation'
)

# Evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size
)

Found 72 images belonging to 4 classes.
Found 8 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [830]:
# Build Model
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))


model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))

model.add(Conv2D(128, (3,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2, padding='same'))



model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_174 (Conv2D)          (None, 78, 78, 32)        896       
_________________________________________________________________
batch_normalization_173 (Bat (None, 78, 78, 32)        128       
_________________________________________________________________
activation_173 (Activation)  (None, 78, 78, 32)        0         
_________________________________________________________________
max_pooling2d_173 (MaxPoolin (None, 39, 39, 32)        0         
_________________________________________________________________
conv2d_175 (Conv2D)          (None, 37, 37, 32)        9248      
_________________________________________________________________
batch_normalization_174 (Bat (None, 37, 37, 32)        128       
_________________________________________________________________
activation_174 (Activation)  (None, 37, 37, 32)      

In [831]:
# Compile model
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [832]:
#Callbacks
# early_stop = EarlyStopping(monitor='val_loss', patience=5)
# lr_redux = ReduceLROnPlateau(monitor='val_loss',
#                             patience = 3, verbose = 1,
#                             factor = 0.1, min_lr = 0.000001)

# log_dir = f"/home/tasos/logs/{int(time.time())}"

# tensorboard = TensorBoard(log_dir=log_dir)
# callbacks = [early_stop, lr_redux, tensorboard]

In [833]:
# Fit model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
    #callbacks=callbacks
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)

print("Test accuracy: {}" .format(test_acc))
print("Test loss: {}" .format(test_loss))

Epoch 1/50
9/9 [==============================] - 2s 129ms/step - loss: 1.4060 - accuracy: 0.4875 - val_loss: 1.3710 - val_accuracy: 0.2500
Epoch 2/50
9/9 [==============================] - 1s 98ms/step - loss: 0.9961 - accuracy: 0.5414 - val_loss: 1.3523 - val_accuracy: 0.6250
Epoch 3/50
9/9 [==============================] - 1s 94ms/step - loss: 0.5972 - accuracy: 0.7986 - val_loss: 1.3858 - val_accuracy: 0.2500
Epoch 4/50
9/9 [==============================] - 1s 94ms/step - loss: 0.3362 - accuracy: 0.8544 - val_loss: 1.4750 - val_accuracy: 0.2500
Epoch 5/50
9/9 [==============================] - 1s 95ms/step - loss: 0.2914 - accuracy: 0.9334 - val_loss: 1.5953 - val_accuracy: 0.2500
Epoch 6/50
9/9 [==============================] - 1s 92ms/step - loss: 0.2193 - accuracy: 0.9738 - val_loss: 1.7134 - val_accuracy: 0.2500
Epoch 7/50
9/9 [==============================] - 1s 93ms/step - loss: 0.0820 - accuracy: 0.9862 - val_loss: 1.7226 - val_accuracy: 0.2500
Epoch 8/50
9/9 [==========

In [841]:
tfjs_target_dir = "C:/Users/younis3/Desktop/train_intel/savedmodels/animals"

model.save('test-model-animals.model', tfjs_target_dir)
tfjs.converters.save_keras_model(model, tfjs_target_dir)


INFO:tensorflow:Assets written to: test-model-animals.model\assets


In [840]:
test_images = []
img_path = 'C:/Users/younis3/Desktop/train_intel/my test/1232222.jpg'
img = np.array(PIL.Image.open(img_path))
img = np.resize(img, (IMG_HEIGHT, IMG_WIDTH, CHANNELS))
img = img.astype('float32')
img /= 255

test_images.append(img)
test_images = np.array(test_images)
#plt.imshow(test_images)

model.predict(test_images)
model.predict_classes(test_images)

array([2], dtype=int64)

In [837]:
  
#    def plot_image(i, predictions_array, true_label, img):
#     true_label, img = true_label[i], img[i]
#     plt.grid(False)
#     plt.xticks([])
#     plt.yticks([])

#     plt.imshow(img)

#     predicted_label = np.argmax(predictions_array)
#     if predicted_label == true_label:
#         color = 'blue'
#     else:
#         color = 'red'

#     plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
#                             100*np.max(predictions_array),
#                             class_names[true_label]),
#                             color=color)

# def plot_value_array(i, predictions_array, true_label):
#     true_label = true_label[i]
#     plt.grid(False)
#     plt.xticks(range(6))
#     plt.yticks([])
#     thisplot = plt.bar(range(6), predictions_array, color="#777777")
#     plt.ylim([0, 1])
#     predicted_label = np.argmax(predictions_array)

#     thisplot[predicted_label].set_color('red')

